In [ ]:
from grav_lens import get_datasets
from grav_lens.models import test_model_image
import matplotlib.pyplot as plt
import numpy as np

import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from os.path import join
home_data = join("..", "data")
max_files = 1000
batch_size = 16

train_dataset, val_dataset, test_dataset = get_datasets(data_index='1', max_files=max_files,
    home=home_data)



Using data folder: ..\data\1


In [ ]:
from utils import CustomScaler

# --- Ejemplo de uso del CustomScaler ---
scaler = CustomScaler()

# Ajustar el scaler a los datos de entrenamiento
scaler.fit_scaler(train_dataset)

# Normalizar un batch de datos
for X_batch, y_batch in train_dataset.take(1):
    y_batch_scaled = scaler.transform(y_batch)
    print("Scaled batch:", y_batch_scaled.shape)

# Desescalar para volver a los valores originales
y_batch_original = scaler.inverse_transform(y_batch_scaled)
print("Original batch:", y_batch_original.shape)


In [ ]:

# Ajustar el scaler a los datos de entrenamiento
scaler.fit_scaler(train_dataset)
# Guardar el scaler ajustado
scaler.save_scaler('scaler.pkl')

# Cargar el scaler cuando lo necesites
scaler.load_scaler('scaler.pkl')

## Transformar con la FFT

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.mixture import GaussianMixture

# Filtro FFT
def fft_filter_image(image, threshold_factor=1):
    """
    Aplica FFT 2D a una imagen, filtra los valores que están por encima del umbral basado en std.
    """
    fft_image = tf.signal.fft2d(tf.cast(image, tf.complex64))
    abs_fft_image = tf.abs(fft_image)
    
    # Calcular estadísticos de la FFT
    mean_value = tf.reduce_mean(abs_fft_image)
    std_value = tf.math.reduce_std(abs_fft_image)
    
    # Filtrar los valores por encima del threshold
    threshold = mean_value + threshold_factor * std_value
    mask = abs_fft_image > threshold
    
    # Aplicar la máscara y filtrar
    filtered_fft = tf.where(mask, fft_image, tf.zeros_like(fft_image))
    
    # Inversa de FFT para reconstruir la imagen filtrada
    filtered_image = tf.signal.ifft2d(filtered_fft)
    
    # Obtener parte real de la imagen filtrada
    return tf.math.real(filtered_image)

# Estadísticas de la imagen
def calculate_image_stats(image):
    mean_value = tf.reduce_mean(image)
    std_value = tf.math.reduce_std(image)
    return mean_value, std_value

# Thresholding
def apply_threshold(image, mean, std, threshold=2.0):
    """
    Aplica un threshold a la imagen para eliminar el ruido.
    """
    return tf.where(image > mean + threshold * std, image, tf.zeros_like(image))

# Ajuste Gaussian Mixture
def fit_gaussian_mixture(image, n_gaussians=5):
    """
    Ajusta un modelo Gaussian Mixture a la imagen.
    """
    # Obtener las coordenadas de los píxeles no cero
    x, y_coords = np.indices(image.shape)
    x = x.flatten()
    y_coords = y_coords.flatten()
    intensities = image.flatten()

    # Crear los datos para el GMM
    data = np.stack((x, y_coords, intensities), axis=1)
    
    # Ajustar un Gaussian Mixture Model (GMM)
    gmm = GaussianMixture(n_components=n_gaussians, covariance_type='full').fit(data)

    return gmm, gmm.means_, gmm.covariances_, gmm.weights_

# Pipeline para procesar la imagen
def process_image_pipeline(image, threshold_factor=1, n_gaussians=5, threshold_value=2.0):
    """
    Ejecuta el pipeline completo: filtro FFT, thresholding, y ajuste de Gaussian Mixture.
    """
    # Paso 1: Filtrar la imagen usando FFT
    filtered_image = fft_filter_image(image, threshold_factor)

    # Paso 2: Calcular estadísticas de la imagen filtrada
    mean_value, std_value = calculate_image_stats(filtered_image)

    # Paso 3: Aplicar un threshold para eliminar ruido
    image_thresholded = apply_threshold(filtered_image, mean_value, std_value, threshold=threshold_value)

    # Paso 4: Ajustar un modelo Gaussian Mixture Model (GMM) a la imagen filtrada y thresholded
    gmm, means, covariances, weights = fit_gaussian_mixture(image_thresholded.numpy(), n_gaussians)

    return {
        'filtered_image': filtered_image,
        'thresholded_image': image_thresholded,
        'gmm': gmm,
        'means': means,
        'covariances': covariances,
        'weights': weights
    }


In [ ]:
# Función para aplicar el pipeline a cada imagen en el dataset
def process_dataset_pipeline(dataset, threshold_factor=1, n_gaussians=5, threshold_value=2.0):
    processed_results = []
    
    for X_batch, y_batch in dataset:
        # Asegurarse de que y_batch tiene la forma (batch, 128, 128, 1)
        for i in range(y_batch.shape[0]):  # Iterar sobre el batch
            image = y_batch[i, :, :, 0]  # Extraer una imagen 2D (128, 128)
            
            # Ejecutar el pipeline de procesamiento
            result = process_image_pipeline(image, threshold_factor, n_gaussians, threshold_value)
            
            # Almacenar el resultado
            processed_results.append(result)
    
    return processed_results

# Aplicar la pipeline al dataset normalizado
processed_train = process_dataset_pipeline(train_dataset_scaled, threshold_factor=1, n_gaussians=5, threshold_value=2.0)

# Puedes hacer lo mismo para val_dataset_scaled y test_dataset_scaled


In [ ]:
# Visualizar la imagen original y filtrada de un ejemplo
import matplotlib.pyplot as plt

def visualize_results(result, original_image):
    plt.figure(figsize=(12, 5))

    # Imagen original
    plt.subplot(1, 3, 1)
    plt.imshow(original_image, cmap='viridis')
    plt.title("Imagen original")
    plt.colorbar()

    # Imagen filtrada
    plt.subplot(1, 3, 2)
    plt.imshow(result['filtered_image'].numpy(), cmap='viridis')
    plt.title("Imagen filtrada (FFT)")
    plt.colorbar()

    # Imagen con thresholding
    plt.subplot(1, 3, 3)
    plt.imshow(result['thresholded_image'].numpy(), cmap='viridis')
    plt.title("Imagen con threshold")
    plt.colorbar()

    plt.show()

# Ejemplo con un resultado del train dataset
example_result = processed_train[0]
original_image = y_batch[0, :, :, 0].numpy()  # Asegúrate de tener la imagen original
visualize_results(example_result, original_image)
